In [1]:
!pip uninstall -y librosa resampy numba
!pip install librosa
!pip install resampy
!pip install soundfile




Found existing installation: librosa 0.10.1
Uninstalling librosa-0.10.1:
  Successfully uninstalled librosa-0.10.1
Found existing installation: resampy 0.4.2
Uninstalling resampy-0.4.2:
  Successfully uninstalled resampy-0.4.2
Found existing installation: numba 0.57.1
Uninstalling numba-0.57.1:
  Successfully uninstalled numba-0.57.1
  Using cached librosa-0.10.1-py3-none-any.whl (253 kB)
  Using cached numba-0.57.1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.6 MB)
  Using cached resampy-0.4.2-py3-none-any.whl (3.1 MB)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.utils import to_categorical
import soundfile as sf



# Define the path to the parent directory containing subfolders 0, 1, 2, ..., 9
parent_path = '/content/drive/MyDrive/data/'


#def load_audio_files(path):
#    X, y = [], []
#    for subdir in os.listdir(path):
#        subdir_path = os.path.join(path, subdir)
#        if os.path.isdir(subdir_path):
#            for filename in os.listdir(subdir_path):
#                if filename.endswith('.wav'):
#                    audio, sr = sf.read(os.path.join(subdir_path, filename))
#
#                    # Continue extracting MFCCs and processing the audio as before...
#                    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40).T, axis=0)
#                    label = int(filename.split('_')[0])  # Extract digit label from filename
#                    X.append(mfccs)
#                    y.append(label)
#    return np.array(X), np.array(y)


def load_audio_files(path, max_length=66150):  # 66150 samples correspond to 1.5 seconds at 44.1kHz
    X, y = [], []
    for subdir in os.listdir(path):
        subdir_path = os.path.join(path, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                if filename.endswith('.wav'):
                    audio, sr = sf.read(os.path.join(subdir_path, filename))

                    # Ensure audio has the same length, either by truncating or zero-padding
                    if len(audio) > max_length:
                        audio = audio[:max_length]
                    elif len(audio) < max_length:
                        audio = np.pad(audio, (0, max_length - len(audio)))

                    mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40).T, axis=0)
                    label = int(filename.split('_')[0])
                    X.append(mfccs)
                    y.append(label)

    return np.array(X), np.array(y)



# Load audio files from subfolders
X, y = load_audio_files(parent_path)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape X for CNN
X_train_cnn = np.expand_dims(X_train, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2)

# Convert labels to one-hot encoding
y_train_cnn = to_categorical(y_train - 1, num_classes=9)
y_test_cnn = to_categorical(y_test - 1, num_classes=9)

# Create a CNN model
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(9, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_cnn, y_train_cnn, epochs=60, batch_size=32, validation_data=(X_test_cnn, y_test_cnn))

# Evaluate the model
score = model.evaluate(X_test_cnn, y_test_cnn, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])



Epoch 1/60
125/125 [==============================] - 2s 8ms/step - loss: 4.3603 - accuracy: 0.2836 - val_loss: 1.3677 - val_accuracy: 0.5882
Epoch 2/60
125/125 [==============================] - 1s 6ms/step - loss: 1.1919 - accuracy: 0.5596 - val_loss: 0.5497 - val_accuracy: 0.7876
Epoch 3/60
125/125 [==============================] - 1s 6ms/step - loss: 0.5875 - accuracy: 0.7723 - val_loss: 0.2428 - val_accuracy: 0.9248
Epoch 4/60
125/125 [==============================] - 1s 6ms/step - loss: 0.3942 - accuracy: 0.8560 - val_loss: 0.1762 - val_accuracy: 0.9399
Epoch 5/60
125/125 [==============================] - 1s 6ms/step - loss: 0.2937 - accuracy: 0.8930 - val_loss: 0.1518 - val_accuracy: 0.9489
Epoch 6/60
125/125 [==============================] - 1s 6ms/step - loss: 0.2261 - accuracy: 0.9163 - val_loss: 0.1135 - val_accuracy: 0.9609
Epoch 7/60
125/125 [==============================] - 1s 6ms/step - loss: 0.1938 - accuracy: 0.9321 - val_loss: 0.1295 - val_accuracy: 0.9579
Epoch 